# <span style="color:#900C3F;">Census Income Data Analysis</span>

### <span style="color:#5E5C5B;">In this notebook, using census income data, we construct various classification models, and implement them above on the data, and analyze the performance of each classification model.</span> 




### <span style="color:#900C3F;">Steps taken:</span> 
1. **Setting up the environment.**
2. **Preparing Data for Analysis.**
3. **Training and Testing.**
4. **Constructing classification models.**
5. **Implementation and Model evaluation.**
6. **Comparing the performance of classification models.**

## <span style="color:#900C3F;">1. Setting up the environment</span>

**Importing requried libraries**

Mainly used:
- **Apache spark** in this project, so we import the some libraries required, **pyspark** for starting a **spark session.** 
- **pandas** for data analysis and manipulation.
- **sklearn** for statistical modeling including classification.




In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from sklearn.model_selection import train_test_split

creating a **spark session** 

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

## <span style="color:#900C3F;">2. Preparing Data for Analysis</span>

**Preparing** dataset taken from http://archive.ics.uci.edu/ml/index.php for the analysis.




In [4]:
#reading the file with spark session's method read.csv, converting the csv file to a dataframe
def readCSVtoDF(csvData):
    return spark.read.csv(csvData)

rawData = 'Data/adult.data'

CensusDF = readCSVtoDF(rawData)

AttributeList = CensusDF.columns

Attributes = {'_c0':'Age', '_c1':'Workclass', '_c2':'Fnlwg', '_c3':'Education', '_c4':'Education Number', 
              '_c5':'Marital Status', '_c6':'Occupation', '_c7':'Relationship', '_c8':'Race', '_c9':'Sex', 
              '_c10':'Capital Gain', '_c11':'Capital Loss', '_c12':'Hours per Week', '_c13':'Native Country', 
              '_c14':'>50K / <=50K'}

#changing the name of attributes with original attribute names
for col in AttributeList:
    CensusDF = CensusDF.withColumnRenamed(col, Attributes[col])
    
#converting dataframe to Pandas Dataframe
def convertDFtoPandas(df):
    return df.toPandas()
PandasDF = convertDFtoPandas(CensusDF)

## <span style="color:#900C3F;">3. Training and Testing</span>

In [ ]:
#splitting data into training and testing(target)
def splitDFtoTestTrain(df):
    X = df.iloc[:,0:14]  
    y = df.iloc[:,14:15]  
splitDFtoTestTrain(PandasDF)

#### Partitioning data with Holdout Method

In [46]:
#partitioning data with Holdout Method
def HoldOutMethod(X, y, testSize, randomState):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state=randomState)
HoldOutMethod(X, y, 0.3, 42)

#### Partitioning data with Cross-Validation Method

#### Partitioning data with  Bagging Ensemble Method

#### Partitioning data with Boosting Ensemble Method

## <span style="color:#900C3F;">4. Constructing classification models</span>

## <span style="color:#900C3F;">5. Implementation and Model evaluation</span>

## <span style="color:#900C3F;">6. Comparing the performance of classification models</span>